In [1]:
import osmnx as ox
import networkx as nx
import geopandas as gpd
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'osmnx'

In [ ]:
towns = ['Bardowick, Bardowick, Germany',
         'Vögelsen, Bardowick, Germany' ,
         'Barum, Bardowick, Germany',
         'Handorf, Bardowick, Germany',
         'Mechtersen, Bardowick, Germany',
         'Radbruch, Bardowick, Germany',
         'Wittorf, Bardowick, Germany',
         
         'Kirchgellersen, Gellersen, Germany',
         'Reppenstedt, Gellersen, Germany',
         'Südergellersen, Gellersen, Germany',
         'Westergellersen, Gellersen, Germany',
         
         'Barnstedt, Ilmenau, Germany',
         'Deutsch Evern, Ilmenau, Germany',
         'Embsen, Ilmenau, Germany',
         'Melbeck, Ilmenau, Germany',
         
         'Barendorf, Ostheide, Germany',
         'Neetze, Ostheide, Germany',
         'Reinstorf, Ostheide, Germany',
         'Thomasburg, Ostheide, Germany',
         'Vastorf, Ostheide, Germany',
         'Wendisch Evern, Ostheide, Germany',
         
         'Artlenburg, Scharnebeck, Germany',
         'Brietlingen, Scharnebeck, Germany',
         'Echem, Scharnebeck, Germany',
         'Hittbergen, Scharnebeck, Germany',
         'Lüdersburg, Scharnebeck, Germany',
         'Rullstorf, Scharnebeck, Germany',
         'Scharnebeck, Scharnebeck, Germany',
        
        'Lüneburg, Germany',
        
        'Adendorf, Germany',
                'Amelinghausen, Amelinghausen, Germany',
         
        'Bleckede, Lüneburg, Germany',
         
        'Dahlenburg, Dahlenburg, Germany']

In [ ]:
#downloading the graph from osm
L = ox.graph_from_place(towns, network_type='drive_service', simplify=True)

In [ ]:
# obtaining bus stops coordinates in the same area
bus_stops = ox.geometries_from_place(towns, {'highway':"bus_stop"})

bus_points = bus_stops[bus_stops.geom_type == "Point"]

# getting osmid values to a column
bus_points.index.name = '#'
bus_points.reset_index(inplace=True)

# dropping duplicate bus stops
unique_stops = bus_points.drop_duplicates(subset=['name'])
unique_stops['osmid'] = ox.distance.nearest_nodes(L, unique_stops.geometry.x, unique_stops.geometry.y)

bus_ids = unique_stops['osmid'].tolist()

In [ ]:
#Calculating distance matrix
distance_matrix = np.zeros(shape=(len(bus_ids),len(bus_ids)))
for i in range(0,len(bus_ids)):
    for j in range(0,len(bus_ids)):
        distance_matrix[i,j]=nx.shortest_path_length(L, bus_ids[i], bus_ids[j], weight='length')

dm = pd.DataFrame(distance_matrix, columns=unique_stops['osmid']).dropna()
dm = dm.set_index(unique_stops['osmid'])

In [ ]:
#saving necessary files
unique_stops.to_file('bus_stops')
dm.to_csv('distance_matrix_large')